# Leetcode 100-199
See https://leetcode.com/problems/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import heapq
from typing import *

In [ ]:
# utility functions and version check

def run_sample(obj, sample, funcname):
    lines=[s for s in sample.splitlines() if len(s)>0 ]
    data=[json.loads(s) for s in lines]
    func=getattr(obj, funcname)
    res=func(*data[:-1])
    if (isinstance(res, float) and not math.isclose(res, data[-1], rel_tol=1e-7, abs_tol=1e-7)) or \
        ((not isinstance(res, float)) and res!=data[-1]):
        raise ValueError(f'expected {data[-1]}, actual {res}')

print(f'python version: {sys.version}')

In [ ]:
# leetcode 101
# https://leetcode.com/problems/symmetric-tree/

sample1='''
[1, [2, 3, 4], [2, 4, 3]]
true
'''

sample2='''
[1, [2, null, 3], [2, null, 3]]
false
'''

sample3='''
[1]
true
'''

sample4='''
[1, [2, 2, null], [2, 2, null]]
false
'''

sample5='''
[5, [4, null, [1, 2, null]], [1, null, [4, 2, null]]]
false
'''

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.children = []

def readtree(ll: List) -> Optional[TreeNode]:
    '''from nested list to tree of TreeNodes, possibly with more than 2 children per node'''
    root=None
    if ll is not None and isinstance(ll, list):
        if len(ll)<1:
            return root
        root=TreeNode(val=ll[0])
        for cl in ll[1:]:
            root.children.append(readtree(cl))
        if len(root.children)>=1:
            root.left=root.children[0]
        if len(root.children)>=2:
            root.right=root.children[1]
    elif ll is not None: # must be a literal
        root=TreeNode(val=ll)
    return root

def writeTree(root: TreeNode) -> List:
    '''from tree of TreeNodes to nested list'''
    if root is None:
        return None
    if root.left is None and root.right is None:
        return root.val
    return [root.val, writeTree(root.left), writeTree(root.right)]

def flattenTree(root: TreeNode) -> List:
    '''from tree of TreeNodes to flattened list (there is no unambiguous reverse op.)'''
    if root is None:
        return [None]
    if root.left is None and root.right is None:
        return [root.val]
    res=flattenTree(root.left)
    res.append(root.val)
    res.extend(flattenTree(root.right))
    return res

def mirrorTree(root: TreeNode) -> TreeNode:
    '''left-right mirror of tree'''
    if root is None:
        return None
    return TreeNode(val=root.val, left=mirrorTree(root.right), right=mirrorTree(root.left))

class Solution:
    def isSymmetric2(self, ll: List) -> bool:
        root=readtree(ll)
        return self.isSymmetric(root)

    # leetcode 101
    def isSymmetric(self, root: Optional[TreeNode]) -> bool:
        if root is None:
            return True
        if root.left is None:
            return root.right is None
        elif root.right is None:
            return False
        listLeft=writeTree(root.left)
        listRightMirror=writeTree(mirrorTree(root.right))
        #print(f'nested {listLeft=} / {listRightMirror=}')
        return listLeft == listRightMirror

run_sample(Solution(), sample1, 'isSymmetric2')
run_sample(Solution(), sample2, 'isSymmetric2')
run_sample(Solution(), sample3, 'isSymmetric2')
run_sample(Solution(), sample4, 'isSymmetric2')
run_sample(Solution(), sample5, 'isSymmetric2')

In [ ]:
# leetcode 102
# https://leetcode.com/problems/binary-tree-level-order-traversal/


sample1='''
[3, 9, [20, 15, 7]]
[[3],[9,20],[15,7]]
'''

sample2='''
[1]
[[1]]
'''

sample3='''
[]
[]
'''

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.children = []

def readtree(ll: List) -> Optional[TreeNode]:
    '''from nested list to tree of TreeNodes, possibly with more than 2 children per node'''
    root=None
    if ll is not None and isinstance(ll, list):
        if len(ll)<1:
            return root
        root=TreeNode(val=ll[0])
        for cl in ll[1:]:
            root.children.append(readtree(cl))
        if len(root.children)>=1:
            root.left=root.children[0]
        if len(root.children)>=2:
            root.right=root.children[1]
    elif ll is not None: # must be a literal
        root=TreeNode(val=ll)
    return root

class Solution:
    def levelOrder2(self, ll: List) -> List[List[int]]:
        root=readtree(ll)
        return self.levelOrder(root)

    # leetcode 102
    def levelOrder(self, root: Optional[TreeNode]) -> List[List[int]]:
        res=[]
        self.levelOrder1(root, res, 0)
        return res

    def levelOrder1(self, root: Optional[TreeNode], res: List[List[int]], depth: int):
        if root is None:
            return
        while len(res)<=depth:
            res.append([])
        res[depth].append(root.val)
        self.levelOrder1(root.left, res, depth+1)
        self.levelOrder1(root.right, res, depth+1)

run_sample(Solution(), sample1, 'levelOrder2')
run_sample(Solution(), sample2, 'levelOrder2')
run_sample(Solution(), sample3, 'levelOrder2')


In [ ]:
# leetcode 103
# https://leetcode.com/problems/binary-tree-zigzag-level-order-traversal/

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    # leetcode 103, idea: based off leetcode 102, needed parts copied here
    def zigzagLevelOrder(self, root: Optional[TreeNode]) -> List[List[int]]:
        res=self.levelOrder(root)
        for i in range(1, len(res), 2):
            res[i]=res[i][::-1]
        return res

    # leetcode 102
    def levelOrder(self, root: Optional[TreeNode]) -> List[List[int]]:
        res=[]
        self.levelOrder1(root, res, 0)
        return res

    def levelOrder1(self, root: Optional[TreeNode], res: List[List[int]], depth: int):
        if root is None:
            return
        while len(res)<=depth:
            res.append([])
        res[depth].append(root.val)
        self.levelOrder1(root.left, res, depth+1)
        self.levelOrder1(root.right, res, depth+1)    

In [ ]:
# leetcode 104
# https://leetcode.com/problems/maximum-depth-of-binary-tree/

sample1='''
[3, 9, [20, 15, 7]]
3
'''

sample2='''
[1, null, 2]
2
'''

sample3='''
[]
0
'''

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.children = []

def readtree(ll: List) -> Optional[TreeNode]:
    root=None
    if ll is not None and isinstance(ll, list):
        if len(ll)<1:
            return root
        root=TreeNode(val=ll[0])
        for cl in ll[1:]:
            root.children.append(readtree(cl))
        if len(root.children)>=1:
            root.left=root.children[0]
        if len(root.children)>=2:
            root.right=root.children[1]
    elif ll is not None: # must be a literal
        root=TreeNode(val=ll)
    return root

class Solution:
    def maxDepth2(self, ll: List) -> int:
        root=readtree(ll)
        return self.maxDepth(root)

    # leetcode 104
    def maxDepth(self, root: Optional[TreeNode]) -> int:
        if root is None:
            return 0
        elif hasattr(root, 'children'):
            return 1+max([self.maxDepth(child) for child in root.children], default=0)
        else:
            return 1+max(self.maxDepth(root.left), self.maxDepth(root.right))

run_sample(Solution(), sample1, 'maxDepth2')
run_sample(Solution(), sample2, 'maxDepth2')
run_sample(Solution(), sample3, 'maxDepth2')

In [ ]:
# leetcode 105
# https://leetcode.com/problems/construct-binary-tree-from-preorder-and-inorder-traversal/

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    # leetcode 105, idea: preorder orders by level, inorder by left-parent-child, so items earlier in preorder than
    # inorder are in left branch, recursively consider root / left subtree / right subtree
    def buildTree(self, preorder: List[int], inorder: List[int]) -> Optional[TreeNode]:
        assert len(preorder)==len(inorder)
        if len(preorder)<1:
            return None
        root_val=preorder[0]
        node=TreeNode(val=root_val)
        i=inorder.index(root_val)
        inorder_left=inorder[:i]
        inorder_left_s=set(inorder_left)
        inorder_right=inorder[i+1:]
        preorder_left=[n for n in preorder if n in inorder_left_s]
        preorder_right=[n for n in preorder if n not in inorder_left_s and n!=root_val]
        node.left=self.buildTree(preorder_left, inorder_left)
        node.right=self.buildTree(preorder_right, inorder_right)
        return node        

In [ ]:
# leetcode 108
# https://leetcode.com/problems/convert-sorted-array-to-binary-search-tree/

sample1='''
[-10,-3,0,5,9]
[0, [-3, -10, null], [9, 5, null]]
'''

sample2='''
[1,3]
[3, 1, null]
'''

sample3='''
[1]
[1]
'''

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def writeTree(root: TreeNode) -> List:
    '''from tree of TreeNodes to nested list'''
    if root is None:
        return None
    if root.left is None and root.right is None:
        return root.val
    return [root.val, writeTree(root.left), writeTree(root.right)]

class Solution:
    def sortedArrayToBST2(self, nums: List[int]) -> List:
        root=self.sortedArrayToBST(nums)
        res=writeTree(root)
        if isinstance(res, int):
            return [res]
        else:
            return res

    # leetcode 108
    def sortedArrayToBST(self, nums: List[int]) -> Optional[TreeNode]:
        if nums is None or len(nums)<1:
            return None
        if len(nums)==1:
            return TreeNode(val=nums[0])
        i=len(nums)//2
        return TreeNode(val=nums[i], left=self.sortedArrayToBST(nums[:i]), right=self.sortedArrayToBST(nums[i+1:]))

run_sample(Solution(), sample1, 'sortedArrayToBST2')
run_sample(Solution(), sample2, 'sortedArrayToBST2')
run_sample(Solution(), sample3, 'sortedArrayToBST2')

In [ ]:
# leetcode 116
# https://leetcode.com/problems/populating-next-right-pointers-in-each-node/

class Node:
    def __init__(self, val: int = 0, left: 'Node' = None, right: 'Node' = None, next: 'Node' = None):
        self.val = val
        self.left = left
        self.right = right
        self.next = next

class Solution:
    # leetcode 116, idea: depth-first traversal with a depth counter and a 'next' per level
    def __init__(self):
        self.connects={} # maps depth to last node, practically constant space

    def connect(self, root: 'Optional[Node]', depth=0) -> 'Optional[Node]':
        if root is None:
            return
        if depth in self.connects:
            self.connects[depth].next=root
        self.connects[depth]=root
        self.connect(root.left, depth+1)
        self.connect(root.right, depth+1)
        return root

In [ ]:
# leetcode 118
# https://leetcode.com/problems/pascals-triangle/

sample1='''
5
[[1],[1,1],[1,2,1],[1,3,3,1],[1,4,6,4,1]]
'''

sample2='''
1
[[1]]
'''

class Solution:
    # leetcode 118
    def generate(self, numRows: int) -> List[List[int]]:
        res=[]
        res.append([1])
        while len(res)<numRows:
            last=res[-1]
            row=[]
            for i in range(len(last)+1):
                if i==0 or i==len(last):
                    row.append(1)
                else:
                    row.append(last[i-1]+last[i])
            res.append(row)
        return res

run_sample(Solution(), sample1, 'generate')
run_sample(Solution(), sample2, 'generate')

In [ ]:
# leetcode 121
# https://leetcode.com/problems/best-time-to-buy-and-sell-stock/
# tags: dynamic

sample1='''
[7,1,5,3,6,4]
5
'''

sample2='''
[7,6,4,3,1]
0
'''

class Solution:
    # leetcode 121, idea: traverse the list, keeping minval and maxprofit compared to current item
    def maxProfit(self, prices: List[int]) -> int:
        minval=None
        maxprofit=0
        for p in prices:
            if minval is not None and p-minval>maxprofit:
                maxprofit=p-minval
            if minval is None or p<minval:
                minval=p
        return maxprofit

run_sample(Solution(), sample1, 'maxProfit')
run_sample(Solution(), sample2, 'maxProfit')

In [ ]:
# leetcode 122
# https://leetcode.com/problems/best-time-to-buy-and-sell-stock-ii/

sample1='''
[7,1,5,3,6,4]
7
'''

sample2='''
[1,2,3,4,5]
4
'''

sample3='''
[7,6,4,3,1]
0
'''

class Solution:
    # leetcode 122, idea: if going up buy, if going down sell
    def maxProfit(self, prices: List[int]) -> int:
        shares=0
        lastbuyprice=None
        profit=0
        prevprice=None
        for price in prices:
            if prevprice is not None:
                if price>prevprice and shares==0:
                    shares=1
                    lastbuyprice=prevprice
                elif price<prevprice and shares==1:
                    shares=0
                    profit+=prevprice-lastbuyprice
                    lastbuyprice=None
            prevprice=price
        # sell what remains at the end (i.e. on the last day)
        if shares==1 and prevprice>lastbuyprice:
            profit+=prevprice-lastbuyprice
        return profit

run_sample(Solution(), sample1, 'maxProfit')
run_sample(Solution(), sample2, 'maxProfit')
run_sample(Solution(), sample3, 'maxProfit')


In [ ]:
# leetcode 125
# https://leetcode.com/problems/valid-palindrome/

sample1='''
"A man, a plan, a canal: Panama"
true
'''

sample2='''
"race a car"
false
'''

sample3='''
" "
true
'''

class Solution:
    # leetcode 125
    def isPalindrome(self, s: str) -> bool:
        i=0
        j=len(s)-1
        while True:
            while i<len(s) and not (('0' <= s[i] <='9') or ('a' <= s[i] <='z') or ('A' <= s[i] <='Z')):
                i+=1
            while j>=0 and not (('0' <= s[j] <='9') or ('a' <= s[j] <='z') or ('A' <= s[j] <='Z')):
                j-=1
            if i>=len(s):
                return j<0
            if j<0:
                return False
            if s[i].lower()!=s[j].lower():
                return False
            i+=1
            j-=1

run_sample(Solution(), sample1, 'isPalindrome')
run_sample(Solution(), sample2, 'isPalindrome')
run_sample(Solution(), sample3, 'isPalindrome')

In [ ]:
# leetcode 136
# https://leetcode.com/problems/single-number/

sample1='''
[2,2,1]
1
'''

sample2='''
[4,1,2,1,2]
4
'''

sample3='''
[1]
1
'''

class Solution:
    # leetcode 136
    def singleNumber(self, nums: List[int]) -> int:
        nums.sort()
        for i in range(0, len(nums), 2):
            if i==len(nums)-1 or nums[i]!=nums[i+1]:
                return nums[i]
        assert False

run_sample(Solution(), sample1, 'singleNumber')
run_sample(Solution(), sample2, 'singleNumber')
run_sample(Solution(), sample3, 'singleNumber')

In [ ]:
# leetcode 141
# https://leetcode.com/problems/linked-list-cycle/

sample1='''
[3,2,0,-4]
1
true
'''

sample2='''
[1,2]
0
true
'''

sample3='''
[1]
-1
false
'''

# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

class Solution:
    def hasCycle2(self, l1: List[int], nnn: int) -> bool:
        y=None
        for n in l1[::-1]:
            x=ListNode(n, y)
            y=x
        l1x=y
        idx={}
        node=l1x
        i=0
        while node is not None:
            idx[i]=node
            i+=1
            node=node.next
        if nnn>=0: # close the loop
            assert nnn<len(idx)
            idx[len(idx)-1].next=idx[nnn]
        res=self.hasCycle(l1x)
        return res

    # leetcode 141
    def hasCycle(self, head: Optional[ListNode]) -> bool:
        if head is None:
            return False
        node=head
        while node is not None:
            if node.val is None:
                return True
            node.val=None
            node=node.next
        return False

run_sample(Solution(), sample1, 'hasCycle2')
run_sample(Solution(), sample2, 'hasCycle2')
run_sample(Solution(), sample3, 'hasCycle2')

In [ ]:
# leetcode 155
# https://leetcode.com/problems/min-stack/

class MinStack:
    # leetcode 155, idea: list with top at the end, cheat on getMin by keeping minimum but if that's removed
    # resort to calling min (could do better with an additional heapq and a set of removed items,
    # but not sure that's needed)
    def __init__(self):
        self.elems=[]
        self.minelem=None

    def push(self, val: int) -> None:
        self.elems.append(val)
        if len(self.elems)<1 or (self.minelem is not None and val<self.minelem):
            self.minelem=val        

    def pop(self) -> None:
        assert len(self.elems)>0
        res=self.elems.pop(-1)
        if self.minelem is not None and res<=self.minelem:
            self.minelem=None
        return res

    def top(self) -> int:
        assert len(self.elems)>0
        return self.elems[-1]

    def getMin(self) -> int:
        assert len(self.elems)>0
        if self.minelem is None:
            self.minelem=min(self.elems)
        return self.minelem

In [ ]:
# leetcode 160
# https://leetcode.com/problems/intersection-of-two-linked-lists/

# no test cases, too much work wrt the custom judge compared to the simple solution below

class Solution:
    # leetcode 160, idea: there are various ok ways to do this; 
    # * the first is a binary search; determine the length from A to end, assume the intersecting node is halfway,
    #   keep refs to that node and the previous one, now traverse from B to the end, if it passes node and previous
    #   the intersection is before prev., if it passes node but not previous then node is the intersecting node, etc.
    # * the second option is a bit more fancy; traverse A to end and count nodes, traverse B to end and count nodes,
    #   now traverse from A to end and reverse the list, now traverse from B to end again - if the lists intersect 
    #   you should now end up at A and have counted nodes from B to A, reverse the list from A's tail back again,
    #   and consider that if a is the segment from A to (but excluding) the intersecting nodes, b same from B, c is
    #   the intersecting segment then now we know a+c and b+c and a+b, so a+c+b+c-(a+b)==2*c, we now know c so also a
    #   and can find the first intersecting node
    # * for now we will cheat, observing that all node values are at least 1
    def getIntersectionNode(self, headA: ListNode, headB: ListNode) -> Optional[ListNode]:
        node=headA # 'reverse'
        while node is not None:
            assert node.val>0
            node.val= -node.val
            node=node.next
        node=headB # find the first marked node from B
        inter=None 
        while node is not None:
            if node.val<0:
                inter=node
                break
            node=node.next
        node=headA # 'reverse' back to normal
        while node is not None:
            node.val= -node.val
            node=node.next
        return inter
        

In [ ]:
# leetcode 162
# https://leetcode.com/problems/find-peak-element/

sample1='''
[1,2,3,1]
2
'''

sample2='''
[1,2,1,3,5,6,4]
1
'''

sample3='''
[-100]
0
'''

class Solution:
    # leetcode 162, interestingly the answer asks for an O(log n) solution (with n max. 1000!!-/),
    # random sampling would probably work, the chances of either the first or last element being
    # a peak are pretty high,
    # if those are ruled out then you would necessarily have to have a peak somewhere in between
    # that you could find via binary search, 
    # let's just go linear for simplicity
    def findPeakElement(self, nums: List[int]) -> int:
        last=None
        for i,n in enumerate(nums):
            if (last is None or last<n) and (i==len(nums)-1 or nums[i+1]<n):
                return i
            last=n
        assert False # there must be a highest element(!)
  
run_sample(Solution(), sample1, 'findPeakElement')
run_sample(Solution(), sample2, 'findPeakElement')
run_sample(Solution(), sample3, 'findPeakElement')

In [ ]:
# unnumbered 'Search for a Range'

sample1='''
[5,7,7,8,8,10]
8
[3,4]
'''

sample2='''
[5,7,7,8,8,10]
6
[-1,-1]
'''

sample3='''
[]
0
[-1,-1]
'''

sample4='''
[9, 9, 9]
9
[0, 2]
'''

sample5='''
[1]
0
[-1, -1]
'''

sample6='''
[2,2]
2
[0, 1]
'''

class Solution:
    # apparently unnumbered 'Search for a Range',
    # idea: classic binary search problem (could search for target-1 and target+1)
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        # search for start of range, last index with val<target
        start=0
        end=len(nums)-1
        if end<start:
            return [-1, -1]
        while start<end-1:
            mid=(start+end)//2
            if nums[mid]>=target:
                end=mid-1
            else:
                start=mid
        mid1=start
        while nums[mid1]<target and mid1+1<len(nums):
            mid1+=1
        # search for end of range, first index with val>target
        start=mid1
        end=len(nums)-1
        if end<start:
            return [-1, -1]
        while start<end-1:
            mid=(start+end)//2
            if nums[mid]>target:
                end=mid
            else:
                start=mid+1
        mid2=end
        while nums[mid2]>target and mid2>0:
            mid2-=1
        if mid2<mid1 or nums[mid1]!=target:
            return [-1, -1]
        return [mid1, mid2]
                    
run_sample(Solution(), sample1, 'searchRange')
run_sample(Solution(), sample2, 'searchRange')
run_sample(Solution(), sample3, 'searchRange')
run_sample(Solution(), sample4, 'searchRange')
run_sample(Solution(), sample5, 'searchRange')
run_sample(Solution(), sample6, 'searchRange')

In [ ]:
# leetcode 163
# https://leetcode.com/problems/missing-ranges/

# premium, not yet available

In [ ]:
# leetcode 166
# https://leetcode.com/problems/fraction-to-recurring-decimal/

sample1='''
1
2
"0.5"
'''

sample2='''
2
1
"2"
'''

sample3='''
4
333
"0.(012)"
'''

sample4='''
355
113
"3.(1415929203539823008849557522123893805309734513274336283185840707964601769911504424778761061946902654867256637168)"
'''

sample5='''
1
333
"0.(003)"
'''

sample6='''
0
-5
"0"
'''

sample7='''
-22
-2
"11"
'''

sample8='''
1
214748364
"0.00(000000465661289042462740251655654056577585848337359161441621040707904997124914069194026549138227660723878669455195477065427143370461252966751355553982241280310754777158628319049732085502639731402098131932683780538602845887105337854867197032523144157689601770377165713821223802198558308923834223016478952081795603341592860749337303449725)"
'''

import decimal
import re
class Solution:
    # leetcode 166, idea: use decimal builtin module to divide out to a great nr of decimals,
    # then find the repeating string, initially using the re module. This is a bit of a nasty puzzle
    # with many edge cases and no clearly defined result.
    # TODO instead of re create something based on https://stackoverflow.com/questions/29481088/how-can-i-tell-if-a-string-repeats-itself-in-python
    # (divquot answer), 25k string result, go to middle, take one char, search to right for char, if found do again with two chars etc.
    # once max length is found try to shift the window to the left as far as possible, done
    def fractionToDecimal(self, numerator: int, denominator: int) -> str:
        if numerator==0:
            return "0"
        decimal.setcontext(decimal.Context(prec=25000))
        x=decimal.Decimal(numerator)/decimal.Decimal(denominator)
        x=f'{x:f}' # no scientific notation
        print('a', len(x), x[:1000])
        if not '.' in x:
            return x
        REPEATER = re.compile(r"^(.*?)(.+)\2+?.*?$")
        match = REPEATER.match(x)
        if not match:
             return x
        preamble=match.group(1)
        repeats=match.group(2)
        print('b', preamble, repeats)
        REPEATER = re.compile(r"^(.+?)\1+$")
        match = REPEATER.match(repeats)
        return preamble+'('+match.group(1)+')' if match else preamble+'('+repeats+')'

run_sample(Solution(), sample1, 'fractionToDecimal')
run_sample(Solution(), sample2, 'fractionToDecimal')
run_sample(Solution(), sample3, 'fractionToDecimal')
run_sample(Solution(), sample4, 'fractionToDecimal')
run_sample(Solution(), sample5, 'fractionToDecimal')
run_sample(Solution(), sample6, 'fractionToDecimal')
run_sample(Solution(), sample7, 'fractionToDecimal')
run_sample(Solution(), sample8, 'fractionToDecimal')

In [ ]:
# leetcode 171
# https://leetcode.com/problems/excel-sheet-column-number/

sample1='''
"A"
1
'''

sample2='''
"AB"
28
'''

sample3='''
"ZY"
701
'''

class Solution:
    # leetcode 171, idea: simply base 26 numbers convert to decimal
    def titleToNumber(self, columnTitle: str) -> int:
        res=0
        for c in columnTitle:
            res=26*res+ord(c)+1-ord('A')
        return res

run_sample(Solution(), sample1, 'titleToNumber')
run_sample(Solution(), sample2, 'titleToNumber')
run_sample(Solution(), sample3, 'titleToNumber')

In [ ]:
# leetcode 172
# https://leetcode.com/problems/factorial-trailing-zeroes/

sample1='''
3
0
'''

sample2='''
5
1
'''

sample3='''
0
0
'''

sample4='''
10000
2499
'''

class Solution:
    # leetcode 172, idea: obviously you're intended to iterate over n, counting occurences of 2, 5
    # and their multiples and take it from there,
    # instead it could be fun to use python's fast factorial; math.factorial and naively count 
    # (this is indeed fast enough, the implementation is C-based)
    def trailingZeroes(self, n: int) -> int:
        if n==0:
            return 0
        s=str(math.factorial(n))
        i=len(s)
        while i>0 and s[i-1]=='0':
            i-=1
        return len(s)-i

run_sample(Solution(), sample1, 'trailingZeroes')
run_sample(Solution(), sample2, 'trailingZeroes')
run_sample(Solution(), sample3, 'trailingZeroes')
run_sample(Solution(), sample4, 'trailingZeroes')

In [ ]:
# leetcode 189
# https://leetcode.com/problems/rotate-array/

sample1='''
[1,2,3,4,5,6,7]
3
[5,6,7,1,2,3,4]
'''

sample2='''
[-1,-100,3,99]
2
[3,99,-1,-100]
'''

sample3='''
[1,2,3,4,5,6]
4
[3,4,5,6,1,2]
'''

sample4='''
[1,2]
0
[1,2]
'''

sample5='''
[1,2,3,4,5,6,7,8,9,10]
4
[7,8,9,10,1,2,3,4,5,6]
'''

sample6='''
[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]
38
[17,18,19,20,21,22,23,24,25,26,27,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
'''

class Solution:
    def rotate2(self, nums: List[int], k: int) -> List[int]:
        self.rotate(nums, k)
        return nums

    # leetcode 189, idea: in-place, pass through values until coming back to original spot
    # (or not, depending on gcd(len(nums), k) )
    def rotate(self, nums: List[int], k: int) -> None:
        k=k%len(nums)
        if k==0:
            return
        gcd=math.gcd(len(nums), k)
        loopsz=(len(nums)+gcd-1)//gcd
        #print(f'{len(nums)=}, {k=}, {gcd=}, {loopsz=}')
        for i0 in range(gcd):
            i=i0
            n=nums[i]
            for x in range(loopsz):
                newindex=(i+k)%len(nums)
                saved=nums[newindex]
                nums[newindex]=n
                n=saved
                i=newindex

run_sample(Solution(), sample1, 'rotate2')
run_sample(Solution(), sample2, 'rotate2')
run_sample(Solution(), sample3, 'rotate2')
run_sample(Solution(), sample4, 'rotate2')
run_sample(Solution(), sample5, 'rotate2')
run_sample(Solution(), sample6, 'rotate2')

In [ ]:
# leetcode 190
# https://leetcode.com/problems/reverse-bits/

sample1='''
43261596
964176192
'''

sample2='''
4294967293
3221225471
'''

sample3='''
0
0
'''

class Solution:
    # leetcode 190
    def reverseBits(self, n: int) -> int:
        n_str=f'{n:b}'
        #print(f'n={n_str}, len={len(n_str)}')
        x=1 # bitmask scanning n
        y=0
        for i in range(32):
            y<<=1
            if n&x:
                y|=1
            x<<=1
        y_str=f'{y:b}'
        #print(f'y={y_str}, len={len(y_str)}')
        return y

run_sample(Solution(), sample1, 'reverseBits')
run_sample(Solution(), sample2, 'reverseBits')
run_sample(Solution(), sample3, 'reverseBits')

In [ ]:
# leetcode 191
# https://leetcode.com/problems/number-of-1-bits/

sample1='''
"00000000000000000000000000001011"
3
'''

sample2='''
"00000000000000000000000010000000"
1
'''

sample3='''
"11111111111111111111111111111101"
31
'''

class Solution:
    def hammingWeight2(self, n: str) -> int:
        return self.hammingWeight(int(n, 2))

    # leetcode 191
    def hammingWeight(self, n: int) -> int:
        count=0
        for i in range(32):
            if n & 1:
                count+=1
            n>>=1
        return count

run_sample(Solution(), sample1, 'hammingWeight2')
run_sample(Solution(), sample2, 'hammingWeight2')
run_sample(Solution(), sample3, 'hammingWeight2')

In [ ]:
# leetcode 198
# https://leetcode.com/problems/house-robber/
# tags: dynamic

sample1='''
[1,2,3,1]
4
'''

sample2='''
[2,7,9,3,1]
12
'''

sample3='''
[1]
1
'''

sample4='''
[9,1,1,3]
12
'''

sample5='''
[4,1,2,7,5,3,1]
14
'''

sample6='''
[2,4,8,9,9,3]
19
'''

class Solution:
    # leetcode 198, idea: at any point decide whether to rob or not, and keep cumulative max score of
    # (robbed this house, did not rob this house)
    def rob(self, nums: List[int]) -> int:
        if len(nums)<1:
            return 0
        cums={} # maps nums index to tuples of (cum-max robbed, cum-max not-robbed)
        for i in range(len(nums)):
            n=nums[i]
            if i==0:
                cums[i]=(n, 0)
            else:
                cums[i]=(n+cums[i-1][1], max(cums[i-1]))
        return max(cums[len(nums)-1])
     
run_sample(Solution(), sample1, 'rob')
run_sample(Solution(), sample2, 'rob')
run_sample(Solution(), sample3, 'rob')
run_sample(Solution(), sample4, 'rob')
run_sample(Solution(), sample5, 'rob')
run_sample(Solution(), sample6, 'rob')